# About the dataset
Attribute Information:

1.	lettr	capital letter	(26 values from A to Z) 
2.	x-box	horizontal position of box	(integer) 
3.	y-box	vertical position of box	(integer) 
4.	width	width of box	(integer) 
5.	high height of box	(integer) 
6.	onpix	total # on pixels	(integer) 
7.	x-bar	mean x of on pixels in box	(integer) 
8.	y-bar	mean y of on pixels in box	(integer) 
9.	x2bar	mean x variance	(integer) 
10.	y2bar	mean y variance	(integer) 
11.	xybar	mean x y correlation	(integer) 
12.	x2ybr	mean of x * x * y	(integer) 
13.	xy2br	mean of x * y * y	(integer) 
14.	x-ege	mean edge count left to right	(integer) 
15.	xegvy	correlation of x-ege with y	(integer) 
16.	y-ege	mean edge count bottom to top	(integer) 
17.	yegvx	correlation of y-ege with x	(integer)

In [49]:
import pandas as pd
import numpy as np

letters = [
    'A',
    'B',
    'C',
    'D',
    'E',
    'F',
    'G',
    'H',
    'I',
    'J',
    'K',
    'L',
    'M',
    'N',
    'O',
    'P',
    'Q',
    'R',
    'S',
    'T',
    'U',
    'V',
    'W',
    'X',
    'Y',
    'Z'
]

letterNums = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]


url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data'
df = pd.read_csv(url, header=None)
df = df.sort_values([0]).rename(index=str, columns={0:"letter", 1:"x-pos", 2:"y-pos", 3:"width", 4:"height", 5:"numPixels", 6:"meanXPixels", 7:"meanYPixels", 8:"meanXVar", 9:"meanYVar", 10:"meanXYCor", 11:"meanXXY", 12:"meanXYY", 13:"meanEdgeCnt", 14:"xEdgCorr", 15:"meanYEdge", 16:"yEdgeCorr"})
df['letter'].replace(letters, letterNums, inplace=True)


# Normalize all values by max
for i in range(16):
    df['x-pos'].replace(i, i / max(df['x-pos']), inplace=True)
    df['y-pos'].replace(i, i / max(df['y-pos']), inplace=True)
    df['width'].replace(i, i / max(df['width']), inplace=True)
    df['height'].replace(i, i / max(df['height']), inplace=True)
    df['numPixels'].replace(i, i / max(df['numPixels']), inplace=True)
    df['meanXPixels'].replace(i, i / max(df['meanXPixels']), inplace=True)
    df['meanYPixels'].replace(i, i / max(df['meanYPixels']), inplace=True)
    df['meanXVar'].replace(i, i / max(df['meanXVar']), inplace=True)
    df['meanYVar'].replace(i, i / max(df['meanYVar']), inplace=True)
    df['meanXYCor'].replace(i, i / max(df['meanXYCor']), inplace=True)
    df['meanXXY'].replace(i, i / max(df['meanXXY']), inplace=True)
    df['meanXYY'].replace(i, i / max(df['meanXYY']), inplace=True)
    df['meanEdgeCnt'].replace(i, i / max(df['meanEdgeCnt']), inplace=True)
    df['xEdgCorr'].replace(i, i / max(df['xEdgCorr']), inplace=True)
    df['meanYEdge'].replace(i, i / max(df['meanYEdge']), inplace=True)
    df['yEdgeCorr'].replace(i, i / max(df['yEdgeCorr']), inplace=True)
                     

letters_input = df.values.tolist()
print(letters_input[0])




[1.0, 0.26666666666666666, 0.6, 0.4, 0.4, 0.13333333333333333, 0.6, 0.3333333333333333, 0.2, 0.06666666666666667, 0.5333333333333333, 0.06666666666666667, 0.5333333333333333, 0.13333333333333333, 0.4666666666666667, 0.13333333333333333, 0.5333333333333333]


In [51]:
from math import exp
from sklearn.metrics import confusion_matrix

learning_rates = [0.5,0.1,0.01,0.001]
epochs = 10

# Calculate neuron activation for input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation


# Transfer neuron activation implements logistic function (sigmoid)
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

# Calculate derivative of neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.0005348048046610517}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6213859615555266, 'delta': -0.14619064683582808}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.6573693455986976, 'delta': 0.0771723774346327}]


In [52]:
expected = [1, 1, 0, 1, 0, 0, 1, 0, 0, 0]
predicted = [1, 0, 0, 1, 0, 0, 1, 1, 1, 0]
results = confusion_matrix(expected, predicted)
print(results)

[[4 2]
 [1 3]]


In [61]:
from sklearn import datasets
from sklearn.metrics import confusion_matrix 
from sklearn.cross_validation import train_test_split

# loading the iris dataset 
iris = datasets.load_iris() 
  
# X -> features, y -> label 
X = iris.data 
y = iris.target 
  
# dividing X, y into train and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0) 
  
# training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
  
# creating a confusion matrix 
cm = confusion_matrix(y_test, dtree_predictions) 
print(cm)

[[13  0  0]
 [ 0 15  1]
 [ 0  3  6]]
